In [2]:
import sqlite3
import pandas as pd

In [3]:
conn = sqlite3.connect('/home/greenman225/Code/github/donkey_golf/donkey_golf/site.db')
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

,name
0,user
1,teams
2,rankings
3,pre_tourney
4,leaderboard_final
5,leaderboard


In [4]:
sql = """
SELECT * FROm leaderboard
"""
pd.read_sql(sql, conn)

,pos,player,to_par,r1,r2,r3,r4,total_strokes,earnings,fedex_pts,tourney_id,thru,donkey_score,missed_cut,player_left
0,1,Tiger Woods,-13,70,68,67,70,275,"$2,070,000.00",600,401056527,F,-13,0,0
1,T2,Dustin Johnson,-12,68,70,70,68,276,"$858,667.00",230,401056527,F,-12,0,0
2,T2,Xander Schauffele,-12,73,65,70,68,276,"$858,667.00",230,401056527,F,-12,0,0
3,T2,Brooks Koepka,-12,66,71,69,70,276,"$858,667.00",230,401056527,F,-12,0,0
4,T5,Jason Day,-11,70,67,73,67,277,"$403,938.00",106,401056527,F,-11,0,0
5,T5,Webb Simpson,-11,72,71,64,70,277,"$403,938.00",106,401056527,F,-11,0,0
6,T5,Tony Finau,-11,71,70,64,72,277,"$403,938.00",106,401056527,F,-11,0,0
7,T5,Francesco Molinari,-11,70,67,66,74,277,"$403,938.00",106,401056527,F,-11,0,0
8,T9,Jon Rahm,-10,69,70,71,68,278,"$310,500.00",82,401056527,F,-10,0,0
9,T9,Patrick Cantlay,-10,73,73,64,68,278,"$310,500.00",82,401056527,F,-10,0,0


In [ ]:
sql = """
SELECT c.id, c.username, b.* FROM teams a
    LEFT JOIN leaderboard b
    ON a.golfer=b.player
    LEFT JOIN user c
    ON a.id=c.id
    LIMIT 10
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT a.id, c.username, sum(b.donkey_score) as donkey_score,
sum(b.missed_cut) as missed_cut 
FROM teams a
    LEFT JOIN leaderboard b
    ON a.golfer=b.player
    LEFT JOIN user c
    ON a.id=c.id
GROUP BY 1,2
"""
pd.read_sql(sql, conn)

In [ ]:
sql = """
SELECT * FROM teams
"""
pd.read_sql(sql, conn)

In [ ]:
user_df = pd.read_sql(sql, conn)

In [ ]:
user_df

In [ ]:
def aggregate_team_score():
    '''
    Pull each users aggregate score for the current tourney
    '''
    
    sql = """
    SELECT c.id, c.username, b.* FROM teams a
    LEFT JOIN leaderboard b 
    ON a.golfer=b.player
    LEFT JOIN user c 
    ON a.id=c.id
    """

    df = pd.read_sql(sql, conn)

    # Convert to par field to an integer so we can aggregate
    df['user_score'] = df['to_par'].str.replace('+','').str.replace('E','0')
    df['user_score'] = df['user_score'].astype(int)

    # Get an overall score for each user
    team_score_df = df.groupby(['id','username'])['user_score'].sum().reset_index()
    team_score_df.sort_values('user_score', inplace=True)
    
    team_score_df['rank'] = team_score_df['user_score'].rank(ascending=True, method='min').astype(int)

    return team_score_df

def determine_ties_in_scoreboard(df):
    '''
    Takes a scoreboard df and determines rank and handles ties
    '''
    df_ties = df.copy()

    # Need to create a better rank that handles ties
    df_ties['new_rank'] = df_ties['rank'].copy()

    df_ties = df_ties.groupby(['rank'])['id'].count().reset_index()
    df_ties.loc[df_ties['id'] > 1, 'new_rank'] = "T" 

    df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)
    df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)
    
    return df_ties[['rank','final_rank']]

def pull_scoreboard():
    '''
    Pulls aggregate game score and then adds in tie info
    '''
    df = pull_game_score()
    tie_df = determine_ties_in_scoreboard(df)
    
    df = df.merge(tie_df,
        how='left',
        on='rank')
    
    return df

In [ ]:
df = pull_game_score()

In [ ]:
pull_scoreboard()

In [ ]:
df.merge(tie_df,
        how='left',
        on='rank')

In [ ]:
df_ties

In [ ]:

#+ df_ties.loc[df_ties['id'] > 1, 'rank'].astype(str)
#df_ties = df_ties[['rank', 'new_rank']].copy()

In [ ]:
df_ties

In [ ]:
df_ties.loc[df_ties['new_rank'].notnull(), 'final_rank'] = 'T' + df_ties['rank'].astype(str)

df_ties.loc[df_ties['new_rank'].isnull(), 'final_rank'] = df_ties['rank'].astype(str)

In [ ]:
df_ties

In [ ]:
df

In [ ]:
pd.read_sql("SELECT * FROM leaderboard", conn)

# Show all tables in our database 

In [ ]:
pd.read_sql('SELECT name FROM sqlite_master WHERE type = "table"', conn)

In [ ]:
user_id=6
sql = f"""
SELECT * from teams a
INNER JOIN leaderboard b
    ON a.golfer = b.player
WHERE id = {user_id}    
"""
pd.read_sql(sql, conn)

In [ ]:
pd.read_sql('SELECT * from leaderboard', conn)

In [ ]:
pd.read_sql('SELECT * from user', conn)